# Risk_transaction_rule update Risk_score_static


In [ ]:
import pandas as pd, numpy as np
import pyodbc
import statistics
import tqdm
from tqdm import tqdm 
tqdm.pandas()
from configparser import ConfigParser

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
riskscore='''UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES SET RISK_SCORE_STATIC=T.RISK_SCORE
FROM
(
SELECT  DISTINCT ID_,R.BATCH_ID,R.REPORT_ID,R.ENTITY_ID,R.ENTITY_TYPE,R.RULE_NO,R.RULE_VALUE,
CASE 
WHEN T.THRESHOLD1_1 IS NULL THEN 0
WHEN R.RULE_VALUE < T.THRESHOLD1_1 THEN 0 
WHEN R.RULE_VALUE = T.THRESHOLD1_1 THEN 1
WHEN R.RULE_VALUE <= T.THRESHOLD1_2 THEN 2
WHEN R.RULE_VALUE <= T.THRESHOLD1_3 THEN 3
WHEN R.RULE_VALUE <= T.THRESHOLD1_4 THEN 4
WHEN R.RULE_VALUE <= T.THRESHOLD1_5 THEN 5
WHEN R.RULE_VALUE <= T.THRESHOLD1_6 THEN 6
WHEN R.RULE_VALUE <= T.THRESHOLD1_7 THEN 7
WHEN R.RULE_VALUE <= T.THRESHOLD1_8 THEN 8
WHEN R.RULE_VALUE <= T.THRESHOLD1_9 THEN 9
WHEN R.RULE_VALUE <= T.THRESHOLD1_10 THEN 10
ELSE 10
END RISK_SCORE,GETDATE() AS CREATE_DATE
FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES R 
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID = R.REPORT_ID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_ENTITY_THRESHOLD T ON R.PEER_GROUP_ID = T.PEER_GROUP_ID 
AND T.ENTITY_TYPE= R.ENTITY_TYPE AND T.RULE_NO=R.RULE_NO
WHERE R.RISK_SCORE_STATIC=0  )T
WHERE T.ID_ IS NOT NULL AND  T.ID_=FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES.ID_'''
RISK_SCORE_1 = '''UPDATE FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES SET RISK_SCORE_STATIC=T.RISK_SCORE
FROM
(
SELECT  DISTINCT ID_,R.BATCH_ID,R.REPORT_ID,R.ENTITY_ID,R.ENTITY_TYPE,R.RULE_NO,R.RULE_VALUE,R.PEER_GROUP_ID,
CASE 
WHEN T.THRESHOLD1_1 IS NULL THEN 0
WHEN R.RULE_VALUE < T.THRESHOLD1_1 THEN 0 
WHEN R.RULE_VALUE = T.THRESHOLD1_1 THEN 1
WHEN R.RULE_VALUE <= T.THRESHOLD1_2 THEN 2
WHEN R.RULE_VALUE <= T.THRESHOLD1_3 THEN 3
WHEN R.RULE_VALUE <= T.THRESHOLD1_4 THEN 4
WHEN R.RULE_VALUE <= T.THRESHOLD1_5 THEN 5
WHEN R.RULE_VALUE <= T.THRESHOLD1_6 THEN 6
WHEN R.RULE_VALUE <= T.THRESHOLD1_7 THEN 7
WHEN R.RULE_VALUE <= T.THRESHOLD1_8 THEN 8
WHEN R.RULE_VALUE <= T.THRESHOLD1_9 THEN 9
WHEN R.RULE_VALUE <= T.THRESHOLD1_10 THEN 10
ELSE 10
END RISK_SCORE,GETDATE() AS CREATE_DATE
FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES R 
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID = R.REPORT_ID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_ENTITY_THRESHOLD T ON R.PEER_GROUP_ID = T.PEER_GROUP_ID AND T.ENTITY_TYPE= R.ENTITY_TYPE AND T.RULE_NO=R.RULE_NO
WHERE R.RISK_SCORE_STATIC=0  )T
WHERE T.ID_ IS NULL 
AND T.REPORT_ID = FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES.REPORT_ID
AND T.ENTITY_ID = FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES.ENTITY_ID
AND T.ENTITY_TYPE = FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES.ENTITY_TYPE
AND T.RULE_NO = FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES.RULE_NO
AND T.PEER_GROUP_ID = FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES.PEER_GROUP_ID'''

In [ ]:
cursor.execute(riskscore)
conn.commit()
cursor.execute(RISK_SCORE_1)
print('DONE')
conn.commit()

In [ ]:
Risk_Transaction_Rules=pd.read_sql_query('''SELECT A.* FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES A (NOLOCK)
                                            INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) 
                                            ON RTI.REPORT_ID = A.REPORT_ID''',conn)

In [ ]:
Risk_Transaction_Rules

# Risk_transaction_Update

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
Risk_transaction = pd.read_sql_query('''
SELECT  T.BATCH_ID,T.REPORT_ID,
(SELECT DISTINCT  REPORT_TYPE FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_SUMMARY_STG S (NOLOCK)
WHERE S.REPORT_ID = T.REPORT_ID) REPORT_TYPE,
ROUND(SUM(T.WEIGHTAGE*MEAN_VAL)/SUM(WEIGHTAGE),0) RISK_SCORE_STATIC
FROM (
SELECT T.BATCH_ID,T.REPORT_ID,PRIORITY_NO,WEIGHTAGE,
CASE WHEN SUM(RISK_SCORE_STATIC) = 0 THEN 0 ELSE SUM(SQUARE(RISK_SCORE_STATIC))/SUM(RISK_SCORE_STATIC) END   MEAN_VAL 
FROM  
(SELECT B.PRIORITY_NO, B.WEIGHTAGE , RULE_VALUE,BATCH_ID,A.REPORT_ID,A.RULE_NO,AVG(RISK_SCORE_STATIC) RISK_SCORE_STATIC
FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_TRANSACTION_RULES A (NOLOCK) 
INNER JOIN [FINCORE_DB].FINCORE_ANALYTICS.RISK_TRANSACTION_INCREMENTAL RTI (NOLOCK) ON RTI.REPORT_ID = A.REPORT_ID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES B (NOLOCK) ON A.RULE_NO = B.RULE_NO
GROUP BY BATCH_ID,A.REPORT_ID,A.RULE_NO,B.PRIORITY_NO, B.WEIGHTAGE ,RULE_VALUE) T
GROUP BY T.BATCH_ID,REPORT_ID,PRIORITY_NO,WEIGHTAGE) T
GROUP BY REPORT_ID,BATCH_ID
''',conn)

In [ ]:
Risk_transaction.drop_duplicates(inplace=True)

In [ ]:
list_data = Risk_transaction.values.tolist()

# Inserting Data Into Risk Transaction

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
for data in list_data:
    cursor.execute(f'''INSERT INTO FINCORE_ANALYTICS.RISK_TRANSACTION (BATCH_ID,REPORT_ID,
                       REPORT_TYPE,RISK_SCORE,CREATE_DATE) VALUES({data[0]},{data[1]},'{data[2]}',{data[3]},GETDATE())''')
    cursor.commit()
conn.close()

In [ ]:
print('Transaction Risk calculated sucessfully')